In [2]:
# Retrieve Text

In [138]:
# -*- coding: utf-8 -*-
"""getVidComments.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1-8ATTMVpKQhhaLzxmxrYpuHg4NmxZFYv
"""

# import libraries 
from apiclient.discovery import build 
from urllib.parse import urlparse
from pathlib import Path


#done in google colab so using drive 
# from google.colab import drive 
#mounting google drive to google colab 
# drive.mount('/content/gdrive')


#importing OS to move between directorys in drive
import os
import time
import re
import pprint 
import json
import csv
import logging
import sys
import pandas as pd
import xlrd

# arguments to be passed to build function 
# Google Developer Key needed for YouTube Data API
DEVELOPER_KEY = "..." # put your Google developer key here

YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

class MemoryCache():
    _CACHE = {}

    def get(self, url):
        return MemoryCache._CACHE.get(url)

    def set(self, url, content):
        MemoryCache._CACHE[url] = content

# creating youtube resource object 
# for interacting with API 
youtube = build(YOUTUBE_API_SERVICE_NAME,  
                     YOUTUBE_API_VERSION, 
            developerKey = DEVELOPER_KEY,
               cache=MemoryCache())


def getVideosIds(path):
    ids = []
    videos = pd.read_excel(path)
    for i, row in videos.iterrows():
        ids.append(get_video_id(row['vid_url']))
    return ids

def getCurrentDirectory():
    return os.getcwd()

def getParentDirectory():
    return str(Path(os.getcwd()).parent)



In [139]:
# A function to get the specific ID from the url link of the youtube 

def get_video_id(the_link):
    vid = re.findall('v(?:=|%3D)([^&%?]+)', urlparse(the_link).query)
    if vid:
        return vid[0]
    the_path = urlparse(the_link).path[1:]
    if ((not the_path) or 
        the_path.startswith('user') or 
        the_path.startswith('shared') or
        the_path.startswith('playlist') or
        the_path.startswith('channel') or 
        the_path.startswith('attribution_link') or 
        the_path.startswith('results') or 
        the_path.startswith('edit') or 
        the_path.startswith('categories') or 
        the_path.startswith('c/') or 
        the_path.startswith('view_play_list')): 
            return None
    if the_path.startswith('embed/'): return the_path[len('embed/'):len('embed/')+11]#typical length of the id
    if the_path.startswith('v/'): return the_path[len('v/'):len('v/')+11]
    
    return the_path

def jsonExists(vid_id, path):
    files_names = [ name for name in os.listdir(path) if name.startswith(vid_id) and not os.path.isdir(os.path.join(path, name)) ]
    return len(files_names) > 0

#results_dir_path is the directory for saving output
#results_file_path is the csv file for inputting that has youtube videos
results_dir_path = 'ResultsComments'

    
    



# Pro is +1, netural is 0 , anti is -1

In [140]:
#retrieves all of the video IDS from the CSV file of YouTube videos

results_file_path = 'Neutral.xlsx'

ids = getVideosIds(results_file_path)

List = []
#for every video ID in the list of IDS
for vid_id in ids[72:]:
    #clears list every iteration of for loop
    List.clear() #List contains the comments for this iteration, in the next iteration

     # if json already fetched for youtube video then skip
    if jsonExists(vid_id, results_dir_path):
        continue

    try:
        #try to retrieve about 50-100 comments for a youTube video using the
        #YouTube data api and the video ID
        list_videos_byid = youtube.commentThreads().list( 
        part = "snippet", videoId=vid_id, order = "time", textFormat="plainText", maxResults = 100
                                           ).execute()
        #appends all of the comments to the List
        List.append(list_videos_byid.get("items", []))

        #if there are more comments to gather
        #use the nextPageToken and a while loop to keep going through
        #the video content until all the comments have been found
        print("Now I'm in",vid_id)
        count = 0
        while 'nextPageToken' in list_videos_byid:
            
            print(count,'.New Thread for',vid_id)
            #We want to retreieve the top comment of each video, along with its view count and like count
            list_videos_byid = youtube.commentThreads().list( 
            part = "snippet", videoId=vid_id, order = "time", textFormat="plainText", 
                pageToken = list_videos_byid['nextPageToken'], maxResults = 100
                                           ).execute()
            #appends all of the comments to the List
            List.append(list_videos_byid.get("items", []))
            count+=1
    except:
        List.append("none")

    #checks to see if the list is empty
    if(List[0] != "none"):
        
        
        #=====================================My Added Code is Here===================================
        # Here, I will not save as JSON File, instead I will replace and make it in a csv file
        fields = ['comment','likeCount']
        rows = []
        for thread in List:
            for dic in thread:
                snippet = dic.get('snippet').get('topLevelComment').get('snippet')
                rows.append([snippet['textDisplay'].encode('unicode-escape').decode('utf-8'),snippet['likeCount']])


        rows = sorted(rows, key = lambda x: x[1],reverse=True)
        # writing to csv file  
        with open('Neutral/'+vid_id + '.csv', 'w',newline="") as csvfile:  
            # creating a csv writer object  
            csvwriter = csv.writer(csvfile, )  

            # writing the fields  
            csvwriter.writerow(fields)  

            # writing the data rows  
            csvwriter.writerows(rows) 
            
        # ================================================================================================
         
        #Save to the json file on a specific folder

#         with open('ResultsComments/'+vid_id + '.json', 'w') as outfile:
#             json.dump(obj=List, indent=4, sort_keys=True, fp = outfile)
    else:
        continue
#         #prints out all the videos to a CSV file that weren't able to have their comments taken
#         with open('NoCommentsVideos.csv', 'w') as outfile:
#             writer = csv.writer(outfile)
#             writer. writerow([vid_id])
            



Now I'm in R6VwI_H1GOk
Now I'm in sBciZFE8lAw
0 .New Thread for sBciZFE8lAw
1 .New Thread for sBciZFE8lAw
2 .New Thread for sBciZFE8lAw
3 .New Thread for sBciZFE8lAw
4 .New Thread for sBciZFE8lAw
5 .New Thread for sBciZFE8lAw
6 .New Thread for sBciZFE8lAw
7 .New Thread for sBciZFE8lAw
8 .New Thread for sBciZFE8lAw
9 .New Thread for sBciZFE8lAw
10 .New Thread for sBciZFE8lAw
11 .New Thread for sBciZFE8lAw
12 .New Thread for sBciZFE8lAw
13 .New Thread for sBciZFE8lAw
14 .New Thread for sBciZFE8lAw
15 .New Thread for sBciZFE8lAw
16 .New Thread for sBciZFE8lAw
17 .New Thread for sBciZFE8lAw
18 .New Thread for sBciZFE8lAw
19 .New Thread for sBciZFE8lAw
20 .New Thread for sBciZFE8lAw
21 .New Thread for sBciZFE8lAw
22 .New Thread for sBciZFE8lAw
23 .New Thread for sBciZFE8lAw
24 .New Thread for sBciZFE8lAw
25 .New Thread for sBciZFE8lAw
26 .New Thread for sBciZFE8lAw
27 .New Thread for sBciZFE8lAw
28 .New Thread for sBciZFE8lAw
29 .New Thread for sBciZFE8lAw
30 .New Thread for sBciZFE8lAw
31 

51 .New Thread for JO-ndBqZORk
52 .New Thread for JO-ndBqZORk
53 .New Thread for JO-ndBqZORk
54 .New Thread for JO-ndBqZORk
55 .New Thread for JO-ndBqZORk
56 .New Thread for JO-ndBqZORk
57 .New Thread for JO-ndBqZORk
58 .New Thread for JO-ndBqZORk
59 .New Thread for JO-ndBqZORk
60 .New Thread for JO-ndBqZORk
61 .New Thread for JO-ndBqZORk
62 .New Thread for JO-ndBqZORk
63 .New Thread for JO-ndBqZORk
64 .New Thread for JO-ndBqZORk
65 .New Thread for JO-ndBqZORk
66 .New Thread for JO-ndBqZORk
67 .New Thread for JO-ndBqZORk
68 .New Thread for JO-ndBqZORk
69 .New Thread for JO-ndBqZORk
70 .New Thread for JO-ndBqZORk
71 .New Thread for JO-ndBqZORk
72 .New Thread for JO-ndBqZORk
73 .New Thread for JO-ndBqZORk
74 .New Thread for JO-ndBqZORk
75 .New Thread for JO-ndBqZORk
76 .New Thread for JO-ndBqZORk
77 .New Thread for JO-ndBqZORk
78 .New Thread for JO-ndBqZORk
79 .New Thread for JO-ndBqZORk
80 .New Thread for JO-ndBqZORk
81 .New Thread for JO-ndBqZORk
82 .New Thread for JO-ndBqZORk
83 .New 

10 .New Thread for 4ylYvNnP1rg
11 .New Thread for 4ylYvNnP1rg
12 .New Thread for 4ylYvNnP1rg
13 .New Thread for 4ylYvNnP1rg
14 .New Thread for 4ylYvNnP1rg
15 .New Thread for 4ylYvNnP1rg
16 .New Thread for 4ylYvNnP1rg
17 .New Thread for 4ylYvNnP1rg
18 .New Thread for 4ylYvNnP1rg
19 .New Thread for 4ylYvNnP1rg
20 .New Thread for 4ylYvNnP1rg
21 .New Thread for 4ylYvNnP1rg
22 .New Thread for 4ylYvNnP1rg
23 .New Thread for 4ylYvNnP1rg
24 .New Thread for 4ylYvNnP1rg
25 .New Thread for 4ylYvNnP1rg
26 .New Thread for 4ylYvNnP1rg
27 .New Thread for 4ylYvNnP1rg
28 .New Thread for 4ylYvNnP1rg
29 .New Thread for 4ylYvNnP1rg
30 .New Thread for 4ylYvNnP1rg
31 .New Thread for 4ylYvNnP1rg
32 .New Thread for 4ylYvNnP1rg
33 .New Thread for 4ylYvNnP1rg
34 .New Thread for 4ylYvNnP1rg
35 .New Thread for 4ylYvNnP1rg
36 .New Thread for 4ylYvNnP1rg
37 .New Thread for 4ylYvNnP1rg
38 .New Thread for 4ylYvNnP1rg
39 .New Thread for 4ylYvNnP1rg
40 .New Thread for 4ylYvNnP1rg
41 .New Thread for 4ylYvNnP1rg
42 .New 

55 .New Thread for x3vcSab13Sk
56 .New Thread for x3vcSab13Sk
57 .New Thread for x3vcSab13Sk
58 .New Thread for x3vcSab13Sk
59 .New Thread for x3vcSab13Sk
60 .New Thread for x3vcSab13Sk
61 .New Thread for x3vcSab13Sk
62 .New Thread for x3vcSab13Sk
63 .New Thread for x3vcSab13Sk
64 .New Thread for x3vcSab13Sk
65 .New Thread for x3vcSab13Sk
66 .New Thread for x3vcSab13Sk
67 .New Thread for x3vcSab13Sk
68 .New Thread for x3vcSab13Sk
69 .New Thread for x3vcSab13Sk
70 .New Thread for x3vcSab13Sk
71 .New Thread for x3vcSab13Sk
72 .New Thread for x3vcSab13Sk
73 .New Thread for x3vcSab13Sk
74 .New Thread for x3vcSab13Sk
75 .New Thread for x3vcSab13Sk
76 .New Thread for x3vcSab13Sk
77 .New Thread for x3vcSab13Sk
78 .New Thread for x3vcSab13Sk
79 .New Thread for x3vcSab13Sk
80 .New Thread for x3vcSab13Sk
81 .New Thread for x3vcSab13Sk
82 .New Thread for x3vcSab13Sk
83 .New Thread for x3vcSab13Sk
84 .New Thread for x3vcSab13Sk
85 .New Thread for x3vcSab13Sk
86 .New Thread for x3vcSab13Sk
87 .New 

197 .New Thread for 1gHbwT_R9t0
198 .New Thread for 1gHbwT_R9t0
199 .New Thread for 1gHbwT_R9t0
200 .New Thread for 1gHbwT_R9t0
201 .New Thread for 1gHbwT_R9t0
202 .New Thread for 1gHbwT_R9t0
203 .New Thread for 1gHbwT_R9t0
204 .New Thread for 1gHbwT_R9t0
205 .New Thread for 1gHbwT_R9t0
206 .New Thread for 1gHbwT_R9t0
207 .New Thread for 1gHbwT_R9t0
208 .New Thread for 1gHbwT_R9t0
209 .New Thread for 1gHbwT_R9t0
210 .New Thread for 1gHbwT_R9t0
211 .New Thread for 1gHbwT_R9t0
212 .New Thread for 1gHbwT_R9t0
213 .New Thread for 1gHbwT_R9t0
214 .New Thread for 1gHbwT_R9t0
215 .New Thread for 1gHbwT_R9t0
216 .New Thread for 1gHbwT_R9t0
217 .New Thread for 1gHbwT_R9t0
218 .New Thread for 1gHbwT_R9t0
219 .New Thread for 1gHbwT_R9t0
220 .New Thread for 1gHbwT_R9t0
221 .New Thread for 1gHbwT_R9t0
222 .New Thread for 1gHbwT_R9t0
223 .New Thread for 1gHbwT_R9t0
224 .New Thread for 1gHbwT_R9t0
225 .New Thread for 1gHbwT_R9t0
226 .New Thread for 1gHbwT_R9t0
227 .New Thread for 1gHbwT_R9t0
228 .New

16 .New Thread for xMDdaNLc8DU
17 .New Thread for xMDdaNLc8DU
18 .New Thread for xMDdaNLc8DU
19 .New Thread for xMDdaNLc8DU
20 .New Thread for xMDdaNLc8DU
21 .New Thread for xMDdaNLc8DU
22 .New Thread for xMDdaNLc8DU
23 .New Thread for xMDdaNLc8DU
24 .New Thread for xMDdaNLc8DU
25 .New Thread for xMDdaNLc8DU
26 .New Thread for xMDdaNLc8DU
27 .New Thread for xMDdaNLc8DU
28 .New Thread for xMDdaNLc8DU
29 .New Thread for xMDdaNLc8DU
30 .New Thread for xMDdaNLc8DU
31 .New Thread for xMDdaNLc8DU
32 .New Thread for xMDdaNLc8DU
33 .New Thread for xMDdaNLc8DU
34 .New Thread for xMDdaNLc8DU
35 .New Thread for xMDdaNLc8DU
36 .New Thread for xMDdaNLc8DU
37 .New Thread for xMDdaNLc8DU
Now I'm in OOlsCY9VvJ4
Now I'm in cxB7tgSfYYI
Now I'm in GMUEUnGFSKs
0 .New Thread for GMUEUnGFSKs
Now I'm in In5xf5DHiM4
Now I'm in 6MAVU9EkMWE
Now I'm in 0IaBq_4Xjfo
0 .New Thread for 0IaBq_4Xjfo
1 .New Thread for 0IaBq_4Xjfo
2 .New Thread for 0IaBq_4Xjfo
3 .New Thread for 0IaBq_4Xjfo
4 .New Thread for 0IaBq_4Xjfo


197 .New Thread for zQGOcOUBi6s
198 .New Thread for zQGOcOUBi6s
Now I'm in NHPeZ08BW6M
0 .New Thread for NHPeZ08BW6M
1 .New Thread for NHPeZ08BW6M
Now I'm in T9Zn0SUbSSI
0 .New Thread for T9Zn0SUbSSI
1 .New Thread for T9Zn0SUbSSI
2 .New Thread for T9Zn0SUbSSI
3 .New Thread for T9Zn0SUbSSI
4 .New Thread for T9Zn0SUbSSI
Now I'm in WQptarOLSBU
0 .New Thread for WQptarOLSBU
1 .New Thread for WQptarOLSBU
2 .New Thread for WQptarOLSBU
3 .New Thread for WQptarOLSBU
4 .New Thread for WQptarOLSBU
5 .New Thread for WQptarOLSBU
6 .New Thread for WQptarOLSBU
7 .New Thread for WQptarOLSBU
8 .New Thread for WQptarOLSBU
9 .New Thread for WQptarOLSBU
10 .New Thread for WQptarOLSBU
11 .New Thread for WQptarOLSBU
12 .New Thread for WQptarOLSBU
13 .New Thread for WQptarOLSBU
14 .New Thread for WQptarOLSBU
15 .New Thread for WQptarOLSBU
16 .New Thread for WQptarOLSBU
17 .New Thread for WQptarOLSBU
18 .New Thread for WQptarOLSBU
19 .New Thread for WQptarOLSBU
20 .New Thread for WQptarOLSBU
21 .New Thread f

249 .New Thread for WQptarOLSBU
250 .New Thread for WQptarOLSBU
251 .New Thread for WQptarOLSBU
252 .New Thread for WQptarOLSBU
253 .New Thread for WQptarOLSBU
254 .New Thread for WQptarOLSBU
255 .New Thread for WQptarOLSBU
256 .New Thread for WQptarOLSBU
257 .New Thread for WQptarOLSBU
258 .New Thread for WQptarOLSBU
259 .New Thread for WQptarOLSBU
260 .New Thread for WQptarOLSBU
261 .New Thread for WQptarOLSBU
Now I'm in pWCLPCP1z2w
Now I'm in FVysEMXMbkc
0 .New Thread for FVysEMXMbkc
1 .New Thread for FVysEMXMbkc
2 .New Thread for FVysEMXMbkc
3 .New Thread for FVysEMXMbkc
4 .New Thread for FVysEMXMbkc
5 .New Thread for FVysEMXMbkc
6 .New Thread for FVysEMXMbkc
Now I'm in o1nYpfyfHOs
Now I'm in a259Pr4Me8E
Now I'm in kzXPbOgaVqs
Now I'm in ye7CtNEUm8M
Now I'm in e3-WOVbhpmo
0 .New Thread for e3-WOVbhpmo
1 .New Thread for e3-WOVbhpmo
2 .New Thread for e3-WOVbhpmo
Now I'm in T6N7hP2U7P8
Now I'm in CValFVFZ4Zc
Now I'm in 6oEtF8FdqpA
0 .New Thread for 6oEtF8FdqpA
1 .New Thread for 6oEtF8

15 .New Thread for VzPD009qTN4
16 .New Thread for VzPD009qTN4
17 .New Thread for VzPD009qTN4
18 .New Thread for VzPD009qTN4
19 .New Thread for VzPD009qTN4
20 .New Thread for VzPD009qTN4
21 .New Thread for VzPD009qTN4
22 .New Thread for VzPD009qTN4
23 .New Thread for VzPD009qTN4
24 .New Thread for VzPD009qTN4
25 .New Thread for VzPD009qTN4
26 .New Thread for VzPD009qTN4
27 .New Thread for VzPD009qTN4
28 .New Thread for VzPD009qTN4
29 .New Thread for VzPD009qTN4
30 .New Thread for VzPD009qTN4
31 .New Thread for VzPD009qTN4
32 .New Thread for VzPD009qTN4
33 .New Thread for VzPD009qTN4
34 .New Thread for VzPD009qTN4
35 .New Thread for VzPD009qTN4
36 .New Thread for VzPD009qTN4
37 .New Thread for VzPD009qTN4
38 .New Thread for VzPD009qTN4
39 .New Thread for VzPD009qTN4
40 .New Thread for VzPD009qTN4
41 .New Thread for VzPD009qTN4
42 .New Thread for VzPD009qTN4
43 .New Thread for VzPD009qTN4
44 .New Thread for VzPD009qTN4
45 .New Thread for VzPD009qTN4
46 .New Thread for VzPD009qTN4
47 .New 

33 .New Thread for WQptarOLSBU
34 .New Thread for WQptarOLSBU
35 .New Thread for WQptarOLSBU
36 .New Thread for WQptarOLSBU
37 .New Thread for WQptarOLSBU
38 .New Thread for WQptarOLSBU
39 .New Thread for WQptarOLSBU
40 .New Thread for WQptarOLSBU
41 .New Thread for WQptarOLSBU
42 .New Thread for WQptarOLSBU
43 .New Thread for WQptarOLSBU
44 .New Thread for WQptarOLSBU
45 .New Thread for WQptarOLSBU
46 .New Thread for WQptarOLSBU
47 .New Thread for WQptarOLSBU
48 .New Thread for WQptarOLSBU
49 .New Thread for WQptarOLSBU
50 .New Thread for WQptarOLSBU
51 .New Thread for WQptarOLSBU
52 .New Thread for WQptarOLSBU
53 .New Thread for WQptarOLSBU
54 .New Thread for WQptarOLSBU
55 .New Thread for WQptarOLSBU
56 .New Thread for WQptarOLSBU
57 .New Thread for WQptarOLSBU
58 .New Thread for WQptarOLSBU
59 .New Thread for WQptarOLSBU
60 .New Thread for WQptarOLSBU
61 .New Thread for WQptarOLSBU
62 .New Thread for WQptarOLSBU
63 .New Thread for WQptarOLSBU
64 .New Thread for WQptarOLSBU
65 .New 

In [141]:
#Deciding which videos doesn't have comments so we don't include them in our code

HaveComment = os.listdir('Neutral')
NotHaveComment = []

ids = getVideosIds(results_file_path)
for idd in ids:
    if idd!=None and (idd+".csv") not in HaveComment:
        NotHaveComment.append([idd])
with open('NoCommentsNeutral.csv', 'w',newline='') as outfile:
            writer = csv.writer(outfile)
            writer. writerows(NotHaveComment)
            